## Imports for Player Verification System

---

- `requests`:  
  To send HTTP requests (used for web scraping Flashscore profiles).

- `bs4 (BeautifulSoup)`:  
  To parse and extract information from HTML pages.

- `cv2 (OpenCV)`:  
  To load, decode, and process images.

- `numpy`:  
  To manipulate arrays and perform numerical operations (e.g., averaging embeddings).

- `insightface.app.FaceAnalysis`:  
  To perform face detection and extract face embeddings.

- `sklearn.metrics.pairwise.cosine_similarity`:  
  To compute similarity between two face embeddings.

- `typing (List, Dict, Optional, Tuple)`:  
  To add type hints for better code readability and validation.

- `os`:  
  To handle file system operations (e.g., saving, deleting temporary files).

- `datetime`:  
  To generate timestamps for verification logs.


In [2]:
import requests
from bs4 import BeautifulSoup
import cv2
import numpy as np
from insightface.app import FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict, Optional, Tuple
import os
from datetime import datetime

## Class: PlayerVerificationSystem

This class verifies the identity of a football player by comparing an official image from Flashscore with user-provided selfies.

---

## `__init__(self)`

- Initializes the face recognition model (`buffalo_l` from InsightFace).
- Prepares the model to work on CPU.
- Sets a default similarity threshold of `0.60` for verification.

---

## `search_player_profile(self, player_name: str) -> Optional[str]`

- Searches for the player's Flashscore profile using DuckDuckGo search.
- Returns the URL of the profile if found, otherwise `None`.

---

## `extract_player_image(self, profile_url: str, player_name: str) -> Optional[str]`

- Fetches the player's Flashscore profile page.
- Extracts the player's official image URL by matching `alt` text or keywords.
- Returns the image URL if successful, otherwise `None`.

---

## `process_image(self, image_path: str, is_url: bool = False) -> Optional[np.ndarray]`

- Loads an image either from a local path or a URL.
- Returns the loaded image as a NumPy array.
- Returns `None` if loading fails.

---

## `get_face_embedding(self, image: np.ndarray) -> Optional[np.ndarray]`

- Detects the face in the image using the prepared model.
- Extracts the normalized face embedding vector.
- Returns the embedding or `None` if no face is found.

---

## `verify_player_identity(self, player_name: str, selfie_paths: List[str], save_log: bool = False) -> Dict`

**Workflow:**
1. Search for the player profile.
2. Extract the official image.
3. Process the official image and extract its embedding.
4. Process the provided selfies and extract their embeddings.
5. Average the selfies' embeddings.
6. Compute cosine similarity between the official and combined selfie embeddings.
7. Decide if the player is verified based on the threshold.
8. Optionally save the verification result in a log file.

**Returns:**  
A dictionary containing success status, similarity score, verification result, official image URL, and number of selfies used.

---

## `_save_verification_log(self, player_name: str, similarity_score: float, is_verified: bool) -> None`

- Saves the verification result (timestamp, player, similarity score, verification decision) into `verification_logs.json`.
- If the file does not exist, it creates one.
- Appends each new verification result.

---


# Initialize the system

In [5]:
class PlayerVerificationSystem:
    def __init__(self):
        # Initialize face analysis model (ArcFace)
        self.face_model = FaceAnalysis(name="buffalo_l", providers=["CPUExecutionProvider"])
        self.face_model.prepare(ctx_id=0, det_size=(640, 640))
        self.similarity_threshold = 0.60  # Can be adjusted based on testing

    def search_player_profile(self, player_name: str) -> Optional[str]:
        """Search for player profile on Flashscore using DuckDuckGo"""
        query = f"{player_name} site:flashscore.com/player"
        url = "https://html.duckduckgo.com/html/"
        
        params = {"q": query}
        headers = {"User-Agent": "Mozilla/5.0"}

        try:
            response = requests.post(url, data=params, headers=headers, timeout=10)
            soup = BeautifulSoup(response.text, "html.parser")
            
            for link in soup.find_all("a", href=True):
                href = link["href"]
                if "flashscore.com/player" in href:
                    return href
        except Exception as e:
            print(f"Search error: {str(e)}")
        
        return None
    
    def extract_player_image(self, profile_url: str, player_name: str) -> Optional[str]:
        """Extract official player image from Flashscore profile"""
        headers = {"User-Agent": "Mozilla/5.0"}
        
        try:
            res = requests.get(profile_url, headers=headers, timeout=10)
            soup = BeautifulSoup(res.text, "html.parser")
            
            # Search for player image
            search_parts = player_name.lower().split()
            for img in soup.find_all("img"):
                src = img.get("src", "")
                alt = img.get("alt", "").lower()
                
                if any(part in alt for part in search_parts) or "player" in alt:
                    if src.startswith("//"):
                        return f"https:{src}"
                    elif src.startswith("http"):
                        return src
        except Exception as e:
            print(f"Image extraction error: {str(e)}")
        
        return None
    
    def process_image(self, image_path: str, is_url: bool = False) -> Optional[np.ndarray]:
        """Load and process image from path or URL"""
        try:
            if is_url:
                headers = {
                    "User-Agent": "Mozilla/5.0",
                    "Referer": "https://www.flashscore.com/"
                }
                resp = requests.get(image_path, headers=headers, timeout=10)
                arr = np.asarray(bytearray(resp.content), dtype=np.uint8)
                img = cv2.imdecode(arr, cv2.IMREAD_COLOR)
            else:
                img = cv2.imread(image_path)
            
            return img
        except Exception as e:
            print(f"Image processing error: {str(e)}")
            return None
    
    def get_face_embedding(self, image: np.ndarray) -> Optional[np.ndarray]:
        """Extract face embedding from image"""
        try:
            faces = self.face_model.get(image)
            if faces:
                return faces[0].embedding
            return None
        except Exception as e:
            print(f"Face detection error: {str(e)}")
            return None
    
    def verify_player_identity(
        self,
        player_name: str,
        selfie_paths: List[str],
        save_log: bool = False
    ) -> Dict:
        """
        Main verification workflow:
        1. Search for player profile
        2. Extract official image
        3. Process selfies and official image
        4. Compare embeddings
        5. Return verification result
        """
        # 1. Find player profile
        profile_url = self.search_player_profile(player_name)
        if not profile_url:
            return {
                "success": False,
                "reason": "Player profile not found",
                "player": player_name,
                "verified": False,
                "similarity_score": 0.0
            }
        
        # 2. Extract official image
        official_img_url = self.extract_player_image(profile_url, player_name)
        if not official_img_url:
            return {
                "success": False,
                "reason": "Official image not found",
                "player": player_name,
                "verified": False,
                "similarity_score": 0.0
            }
        
        # 3. Process official image
        official_img = self.process_image(official_img_url, is_url=True)
        if official_img is None:
            return {
                "success": False,
                "reason": "Failed to process official image",
                "player": player_name,
                "verified": False,
                "similarity_score": 0.0
            }
        
        official_embedding = self.get_face_embedding(official_img)
        if official_embedding is None:
            return {
                "success": False,
                "reason": "No face detected in official image",
                "player": player_name,
                "verified": False,
                "similarity_score": 0.0
            }
        
        # 4. Process and combine selfies
        valid_selfie_embeddings = []
        for path in selfie_paths:
            selfie_img = self.process_image(path)
            if selfie_img is None:
                continue
                
            selfie_embedding = self.get_face_embedding(selfie_img)
            if selfie_embedding is not None:
                valid_selfie_embeddings.append(selfie_embedding)
        
        if len(valid_selfie_embeddings) < 1:
            return {
                "success": False,
                "reason": "No valid selfies provided",
                "player": player_name,
                "verified": False,
                "similarity_score": 0.0
            }
        
        # Average all selfie embeddings
        combined_embedding = np.mean(valid_selfie_embeddings, axis=0)
        
        # 5. Calculate similarity
        similarity_score = cosine_similarity(
            official_embedding.reshape(1, -1),
            combined_embedding.reshape(1, -1))
        similarity_score = float(similarity_score[0][0])
        
        is_verified = similarity_score >= self.similarity_threshold
        
        # Save verification log if requested
        if save_log:
            self._save_verification_log(
                player_name,
                similarity_score,
                is_verified
            )
        
        return {
            "success": True,
            "player": player_name,
            "verified": is_verified,
            "similarity_score": similarity_score,
            "official_image_url": official_img_url,
            "selfies_used": len(valid_selfie_embeddings),
            "threshold": self.similarity_threshold
        }
    
    def _save_verification_log(
        self,
        player_name: str,
        similarity_score: float,
        is_verified: bool
    ) -> None:
        """Save verification log to file"""
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "player": player_name,
            "similarity_score": similarity_score,
            "verified": is_verified,
            "threshold": self.similarity_threshold
        }
        
        log_file = "verification_logs.json"
        
        # Create file if it doesn't exist
        if not os.path.exists(log_file):
            with open(log_file, "w") as f:
                f.write("[]")
        
        # Append new log entry
        try:
            with open(log_file, "r+") as f:
                logs = json.load(f)
                logs.append(log_entry)
                f.seek(0)
                json.dump(logs, f, indent=2)
        except Exception as e:
            print(f"Failed to save log: {str(e)}")

In [6]:
verifier = PlayerVerificationSystem()
# Initialize empty results list
sample_results = []


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\bouaz/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\bouaz/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\bouaz/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\bouaz/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\bouaz/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.

# Autogenerated verification script for player identity evaluation
 Note: File paths are assumed to match player2 last names as shown in the directory structure

In [8]:
# Print all results
for res in sample_results:
    print(res)


In [22]:
# Example: verify Raum using downloaded selfies

selfie_paths = [
    r"C:\Users\bouaz\Downloads\Raum1.jpg",
    r"C:\Users\bouaz\Downloads\Raum2.jpg",
    r"C:\Users\bouaz\Downloads\Raum3.jpg"
]

verification_system = PlayerVerificationSystem()
result = verification_system.verify_player_identity(
    player_name="David Raum",
    selfie_paths=selfie_paths,
    save_log=True
)

print(result)


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\bouaz/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\bouaz/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\bouaz/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\bouaz/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\bouaz/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.

C:\Users\bouaz\anaconda3\Lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


{'success': True, 'player': 'David Raum', 'verified': False, 'similarity_score': 0.5623358488082886, 'official_image_url': 'https://static.flashscore.com/res/image/data/CG6ZWVXg-KptDiOE0.png', 'selfies_used': 1, 'threshold': 0.6}


In [18]:
sample_results = []

# Verify Benjamin Sesko
result = verifier.verify_player_identity(
    "Benjamin Sesko",
    [
        "C:/Users/bouaz/Downloads/Patrik Schick1.jpg",
        "C:/Users/bouaz/Downloads/Patrik Schick2.jpg",
        "C:/Users/bouaz/Downloads/Patrik Schick3.jpg",
    ],
    save_log=True
)
sample_results.append({
    "player": "Benjamin Sesko",
    "similarity": result["similarity_score"],
    "verified": result["verified"],
    "ground_truth": False
})
print(result)

# Verify Jonathan Clauss
result = verifier.verify_player_identity(
    "Jonathan Clauss",
    [
        "C:/Users/bouaz/Downloads/Thomas Meunier1.jpg",
        "C:/Users/bouaz/Downloads/Thomas Meunier2.jpg",
        "C:/Users/bouaz/Downloads/Thomas Meunier3.jpg",
    ],
    save_log=True
)
sample_results.append({
    "player": "Jonathan Clauss",
    "similarity": result["similarity_score"],
    "verified": result["verified"],
    "ground_truth": False
})
print(result)

# Verify Gonçalo Inácio
result = verifier.verify_player_identity(
    "Gonçalo Inácio",
    [
        "C:/Users/bouaz/Downloads/Djalo.jpg",
        "C:/Users/bouaz/Downloads/Djalo1.jpg",
        "C:/Users/bouaz/Downloads/Djalo2.jpg",
    ],
    save_log=True
)
sample_results.append({
    "player": "Gonçalo Inácio",
    "similarity": result["similarity_score"],
    "verified": result["verified"],
    "ground_truth": False
})
print(result)

# Verify Zeki Celik
result = verifier.verify_player_identity(
    "Zeki Celik",
    [
        "C:/Users/bouaz/Downloads/Soyuncu.jpg",
        "C:/Users/bouaz/Downloads/Soyuncu1.jpg",
        "C:/Users/bouaz/Downloads/Soyuncu2.jpg",
    ],
    save_log=True
)
sample_results.append({
    "player": "Zeki Celik",
    "similarity": result["similarity_score"],
    "verified": result["verified"],
    "ground_truth": False
})
print(result)

# Verify Cody Gakpo
result = verifier.verify_player_identity(
    "Cody Gakpo",
    [
        "C:/Users/bouaz/Downloads/Mateta.jpg",
        "C:/Users/bouaz/Downloads/Mateta1.jpg",
        "C:/Users/bouaz/Downloads/Mateta2.jpg",
    ],
    save_log=True
)
sample_results.append({
    "player": "Cody Gakpo",
    "similarity": result["similarity_score"],
    "verified": result["verified"],
    "ground_truth": False
})
print(result)

# Verify Donyell Malen
result = verifier.verify_player_identity(
    "Donyell Malen",
    [
        "C:/Users/bouaz/Downloads/Boadu.jpg",
        "C:/Users/bouaz/Downloads/Boadu1.jpg",
        "C:/Users/bouaz/Downloads/Boadu2.jpg",
    ],
    save_log=True
)
sample_results.append({
    "player": "Donyell Malen",
    "similarity": result["similarity_score"],
    "verified": result["verified"],
    "ground_truth": False
})
print(result)

# Verify Luca Netz
result = verifier.verify_player_identity(
    "Luca Netz",
    [
        "C:/Users/bouaz/Downloads/Raum.jpg",
        "C:/Users/bouaz/Downloads/Raum1.jpg",
        "C:/Users/bouaz/Downloads/Raum2.jpg",
    ],
    save_log=True
)
sample_results.append({
    "player": "Luca Netz",
    "similarity": result["similarity_score"],
    "verified": result["verified"],
    "ground_truth": False
})
print(result)

# Verify Josip Stanisic
result = verifier.verify_player_identity(
    "Josip Stanisic",
    [
        "C:/Users/bouaz/Downloads/Sosa.jpg",
        "C:/Users/bouaz/Downloads/Sosa1.jpg",
        "C:/Users/bouaz/Downloads/Sosa2.jpg",
    ],
    save_log=True
)
sample_results.append({
    "player": "Josip Stanisic",
    "similarity": result["similarity_score"],
    "verified": result["verified"],
    "ground_truth": False
})
print(result)

{'success': False, 'reason': 'Player profile not found', 'player': 'Benjamin Sesko', 'verified': False, 'similarity_score': 0.0}
{'success': False, 'reason': 'Player profile not found', 'player': 'Jonathan Clauss', 'verified': False, 'similarity_score': 0.0}
{'success': False, 'reason': 'Player profile not found', 'player': 'Gonçalo Inácio', 'verified': False, 'similarity_score': 0.0}
{'success': False, 'reason': 'Player profile not found', 'player': 'Zeki Celik', 'verified': False, 'similarity_score': 0.0}
{'success': False, 'reason': 'Player profile not found', 'player': 'Cody Gakpo', 'verified': False, 'similarity_score': 0.0}
{'success': False, 'reason': 'Player profile not found', 'player': 'Donyell Malen', 'verified': False, 'similarity_score': 0.0}
{'success': False, 'reason': 'Player profile not found', 'player': 'Luca Netz', 'verified': False, 'similarity_score': 0.0}
{'success': False, 'reason': 'Player profile not found', 'player': 'Josip Stanisic', 'verified': False, 'simil

#  Save combined data

In [24]:
import os
import pandas as pd

# Suppose your results are in this list
# sample_results = [...]  # your list of dicts

# 1. Get the Downloads folder
downloads_folder = os.path.join(os.path.expanduser("~"), "Downloads")

# 2. Define the output file path
csv_output_file = os.path.join(downloads_folder, "verification_results1.csv")

# 3. Create the DataFrame from current session
df_new = pd.DataFrame(sample_results)

# 4. Check if the file already exists
if os.path.exists(csv_output_file):
    # 4.a If file exists, load existing data
    df_existing = pd.read_csv(csv_output_file)

    # 4.b Append new data to existing
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)

    # 4.c Save combined data
    df_combined.to_csv(csv_output_file, index=False, encoding="utf-8-sig")
    print(f"✅ Appended results to existing file at {csv_output_file}")

else:
    # 4.d If file doesn't exist, create it
    df_new.to_csv(csv_output_file, index=False, encoding="utf-8-sig")
    print(f"✅ Created new results file at {csv_output_file}")



✅ Appended results to existing file at C:\Users\bouaz\Downloads\verification_results1.csv


In [38]:
import os
import re
import csv
import time
from collections import defaultdict

# Path to your selfies folder
selfies_folder = r"C:\Users\bouaz\Downloads"

# Path to your CSV sheet
csv_sheet = r"C:\Users\bouaz\Downloads\batch_verification_results.csv"

# Initialize verification system
verification_system = PlayerVerificationSystem()

# Read player names from your CSV
player_list = []
with open(csv_sheet, mode="r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        player_list.append(row["player"])  # Assuming "player" column contains the player names

# Group selfies by player name from the Downloads folder
player_selfies = defaultdict(list)

for filename in os.listdir(selfies_folder):
    if filename.lower().endswith(".jpg"):
        match = re.match(r"([A-Za-z]+)", filename)
        if match:
            player_name = match.group(1)
            player_selfies[player_name].append(os.path.join(selfies_folder, filename))

# Sort selfies for each player
for player in player_selfies:
    player_selfies[player] = sorted(player_selfies[player])

# Prepare results
verification_results = []

# Counter for the reboot cycle
players_processed = 0

# Verify each player
for player_name in player_list:
    selfies = player_selfies.get(player_name)
    
    if selfies is None or len(selfies) < 3:
        print(f"Skipping {player_name}: not enough selfies available.")
        continue

    selfies_to_use = selfies[:3]
    print(f"Verifying {player_name} with selfies: {selfies_to_use}")

    result = verification_system.verify_player_identity(
        player_name=player_name,
        selfie_paths=selfies_to_use,
        save_log=True
    )

    verification_results.append(result)

    players_processed += 1

    # After every 3 players, stop and ask for reboot
    if players_processed % 3 == 0:
        print("\n🔵 Please reboot your clé de connexion (Internet key) to get a new IP address.")
        input("🔵 Press ENTER when ready to continue...")
        print("✅ Continuing verification...\n")
        time.sleep(3)  # small delay for safety

# Save all results into a new CSV
output_csv = os.path.join(selfies_folder, "final_batch_verification_results.csv")

with open(output_csv, mode="w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=[
        "player", "verified", "similarity_score", "threshold", "success", "reason", "official_image_url", "selfies_used"
    ])
    writer.writeheader()
    for res in verification_results:
        writer.writerow({
            "player": res.get("player", ""),
            "verified": res.get("verified", False),
            "similarity_score": res.get("similarity_score", 0.0),
            "threshold": res.get("threshold", 0.68),
            "success": res.get("success", False),
            "reason": res.get("reason", ""),
            "official_image_url": res.get("official_image_url", ""),
            "selfies_used": res.get("selfies_used", 0)
        })

print(f"\n✅ All players processed. Final results saved to: {output_csv}")


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\bouaz/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\bouaz/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\bouaz/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\bouaz/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\bouaz/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.

🔵 Press ENTER when ready to continue... 


✅ Continuing verification...

Verifying Cucurella with selfies: ['C:\\Users\\bouaz\\Downloads\\Cucurella.jpg', 'C:\\Users\\bouaz\\Downloads\\Cucurella1.jpg', 'C:\\Users\\bouaz\\Downloads\\Cucurella2.jpg']
Search error: HTTPSConnectionPool(host='html.duckduckgo.com', port=443): Max retries exceeded with url: /html/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000013EAD0AB2F0>, 'Connection to html.duckduckgo.com timed out. (connect timeout=10)'))
Verifying Cullen with selfies: ['C:\\Users\\bouaz\\Downloads\\Cullen.jpg', 'C:\\Users\\bouaz\\Downloads\\Cullen1.jpg', 'C:\\Users\\bouaz\\Downloads\\Cullen2.jpg']
Search error: HTTPSConnectionPool(host='html.duckduckgo.com', port=443): Max retries exceeded with url: /html/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000013EAD0E8E60>, 'Connection to html.duckduckgo.com timed out. (connect timeout=10)'))
Verifying Dani with selfies: ['C:\\Users\\bouaz\\Downloads\\Dani García1.j

C:\Users\bouaz\anaconda3\Lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4



🔵 Please reboot your clé de connexion (Internet key) to get a new IP address.


...

✅ Continuing verification...

Verifying Thauvin with selfies: ['C:\\Users\\bouaz\\Downloads\\Thauvin.jpg', 'C:\\Users\\bouaz\\Downloads\\Thauvin1.jpg', 'C:\\Users\\bouaz\\Downloads\\Thauvin2.jpg']
Verifying Thomasson with selfies: ['C:\\Users\\bouaz\\Downloads\\Thomasson.jpg', 'C:\\Users\\bouaz\\Downloads\\Thomasson1.jpg', 'C:\\Users\\bouaz\\Downloads\\Thomasson2.jpg']
Verifying Wahi with selfies: ['C:\\Users\\bouaz\\Downloads\\Wahi.jpg', 'C:\\Users\\bouaz\\Downloads\\Wahi1.jpg', 'C:\\Users\\bouaz\\Downloads\\Wahi2.jpg']

🔵 Please reboot your clé de connexion (Internet key) to get a new IP address.


🔵 Press ENTER when ready to continue... 


✅ Continuing verification...

Verifying White with selfies: ['C:\\Users\\bouaz\\Downloads\\White.jpg', 'C:\\Users\\bouaz\\Downloads\\White1.jpg', 'C:\\Users\\bouaz\\Downloads\\White2.jpg']


C:\Users\bouaz\anaconda3\Lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4



✅ All players processed. Final results saved to: C:\Users\bouaz\Downloads\final_batch_verification_results.csv


In [40]:
import csv

# Path to your input file
input_csv = r"C:\Users\bouaz\Downloads\final_batch_verification_results.csv"

# Path to save the filtered output
output_csv = r"C:\Users\bouaz\Downloads\filtered_successful_verifications.csv"

# Read and filter
working_players = []

with open(input_csv, mode="r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        if (
            row.get("success", "").lower() == "true" and
            row.get("selfies_used", "0") != "0" and
            row.get("official_image_url", "").strip() != ""
        ):
            working_players.append(row)

# Save only the working ones
with open(output_csv, mode="w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=working_players[0].keys())
    writer.writeheader()
    writer.writerows(working_players)

print(f"✅ Filtered {len(working_players)} working players into: {output_csv}")


✅ Filtered 25 working players into: C:\Users\bouaz\Downloads\filtered_successful_verifications.csv
